### TO DO
- Change GPU to extra large over lunch
- ReadMe on Matterport
- Read : train_shapes, config to understand parameters adnd the shape of the output.
- Code dataframe function for model analysis

Re-run the first cell if you have re-initialised your server.

In [ ]:
#!pip install keras
#!pip install opencv-python

In [ ]:
#!pip install tensorflow==1.15.3 
! pip show tensorflow

In [ ]:
#! pip install keras==2.0.8
! pip show keras

Run all necessary imports:

In [ ]:
import mrcnn
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn import visualize
import skimage
import numpy as np
import colorsys
import argparse
import json
import random
import cv2
import os
from sklearn.model_selection import train_test_split
import glob
import shutil
import pprint
from matplotlib import pyplot
from matplotlib.patches import Rectangle
%matplotlib inline
random.seed(42)

In [ ]:
ROOT_DIR = os.path.abspath("../../")
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
WEIGHTS_PATH = os.path.join(ROOT_DIR, "notebooks/Mask_RCNN/mask_rcnn_coco.h5")

Initialise the image and label paths for train:

In [ ]:
image_paths = sorted(glob.glob("../../inputs/train/images/*jpg"))
label_paths = sorted(glob.glob("../../inputs/train/labels/*json"))

### Configuration

We define a custom configuration for our construction Mask-RCNN:

In [ ]:
class ConstructionConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "construction"
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 50

    # Skip detections with < 70% confidence
    DETECTION_MIN_CONFIDENCE = 0.7

In [ ]:
config = ConstructionConfig()

Let's check our model configuration:

Bonnes metrics ? MAP / IoU

In [ ]:
config.display()

In [ ]:
CONSTRUCTION_DIR = "../../inputs"

### Train and validation set.

First, create your train and validation folders and subfolders:

In [ ]:
def is_split_data():
    """
    Returns True if the data has already been split, False otherwise.
    """
    TRAIN_FOLDER_EXISTS = ("train" in os.listdir("../../inputs"))
    VAL_FOLDER_EXISTS = ("val" in os.listdir("../../inputs"))  
    
    if TRAIN_FOLDER_EXISTS or VAL_FOLDER_EXISTS:
        TRAIN_OR_VAL_HAS_ELEMENTS = (len(os.listdir("../../inputs/train/images")) > 0) or (len(os.listdir("../../inputs/train/labels")) > 0)
        print(f"There are {len(os.listdir('../../inputs/train/images'))} images in train.")
        print(f"There are {len(os.listdir('../../inputs/val/images'))} images in val.")
        if TRAIN_OR_VAL_HAS_ELEMENTS:
            return True
        else:
            print("Folders exist but are empty.")
            print(f"There are {len(os.listdir('../../inputs/images'))} images in the root folder.")
            print(f"There are {len(os.listdir('../../inputs/labels'))} labels in the root folder.")
            return False
    else:
        print("There are no train or val folders.")
        print(f"There are {len(os.listdir('../../inputs/images'))} images in the root folder.")
        print(f"There are {len(os.listdir('../../inputs/labels'))} labels in the root folder.")
        return False

In [ ]:
is_split_data()

This cell moves all your images and labels back to their root folders if they have been split in the past

In [ ]:
if is_split_data():
    ! mv ../../inputs/train/labels/* ../../inputs/labels
    ! mv ../../inputs/train/images/*  ../../inputs/images
    ! mv ../../inputs/val/labels/* ../../inputs/labels
    ! mv ../../inputs/val/images/* ../../inputs/images
    print(f"There are now {len(os.listdir('../../inputs/images'))} images in the root folder.")
    print(f"There are now {len(os.listdir('../../inputs/labels'))} labels in the root folder.")

If the folders do not exist, this cell will create the train and val directories:

In [ ]:
TRAIN_FOLDER_EXISTS = ("train" in os.listdir("../../inputs"))
VAL_FOLDER_EXISTS = ("val" in os.listdir("../../inputs"))

if TRAIN_FOLDER_EXISTS == False or VAL_FOLDER_EXISTS == False:
    ! mkdir "../../inputs/train" "../../inputs/val"
    ! mkdir "../../inputs/train/images" "../../inputs/train/labels"
    ! mkdir "../../inputs/val/images" "../../inputs/val/labels" 

In [ ]:
def train_validation_set(train_percent = 0.8):
    """
    Create a random train and validation split. 
    """
    INPUTS_DIR = "../../inputs"
    TRAIN_DIR = "../../inputs/train/"
    VAL_DIR = "../../inputs/val/"

    # Retrieve all image and label paths
    image_paths = sorted(glob.glob(os.path.join(INPUTS_DIR, "images/*jpg")))
    label_paths = sorted(glob.glob(os.path.join(INPUTS_DIR, "labels/*json")))
    n = len(image_paths)
    train_nb = np.floor(train_percent*n).astype(int)
    indices = random.sample(range(n), train_nb)
    train_image_paths, train_label_paths = [image_paths[i] for i in indices], [label_paths[i] for i in indices]
    valid_image_paths, valid_label_paths = [image_paths[i] for i in range(n-1) if i not in indices], [label_paths[i] for i in range(n-1) if i not in indices]
    for image_path in train_image_paths:
        shutil.move(image_path, TRAIN_DIR+"images")
    for image_path in valid_image_paths:
        shutil.move(image_path, VAL_DIR+"images")
    for label_path in train_label_paths:
        shutil.move(label_path, TRAIN_DIR+"labels")
    for label_path in valid_label_paths:
        shutil.move(label_path, VAL_DIR+"labels")

Instantiate train_validation split:

In [ ]:
train_validation_set(train_percent = 0.8)

In [ ]:
print(f"There are {len(os.listdir('../../inputs/train/images'))} images in train.")
print(f"There are {len(os.listdir('../../inputs/val/images'))} images in val.")
print(f"There are {len(os.listdir('../../inputs/images'))} images in the root folder.")
print(f"There are {len(os.listdir('../../inputs/labels'))} labels in the root folder.")

### Creating the Construction Dataset object.

Loads and pre-processes the data to feed the model. 

Let us define some support functions for this notebook. (they should later on be added to utils, and imported).

In [ ]:
# Some utils functions we define locally

def load_labels_from_folder(label_paths):
    """
    Load and pre-format labels to feed to ConstructionDataset. 
    folder: folder to retrieve the labels from
    """
    labels = list()
    for label_path in label_paths:
        with open(label_path) as json_data:
            lbl = reformat_label(json.load(json_data), label_path)
            if lbl is not None:
                labels.append(lbl)
    return labels

def reformat_label(label, label_path):
    """
    Reformat label to correspond to VGG annotator standard. 
    """
    objects = label["objects"]
    filename = os.path.split(label_path)[-1] # retrieve filename from full path
    object_list = []
    objects = list(filter(is_polygon, objects))
    for i in range(len(objects)):
        object_list.append(reshape_poly_coordinates(objects[i]))
    return {
            'filename': filename,
             'objects': {"region_attributes" : {},
                        "shape_attributes" : object_list},
             'size': label["size"]
            }

def is_polygon(obj):
    """
    Boolean determining whether an object is a polygon or not. 
    obj: a label pertaining to an image
    """
    if obj['geometryType'] == 'polygon':
        return True
    else:
        return False
    
def reshape_poly_coordinates(object_label):
    """
    Reshape polygons points to generate the mask. 
    """
    ext_points = object_label["points"]["exterior"]
    # TO DO - find how to fix interior points
    return {
            "all_points_x": [x[0] for x in ext_points],
            "all_points_y" : [x[1] for x in ext_points],
            "name":"polygon",
            "class": object_label["classTitle"]
        }

def filter_on_image_id(image_info_list,image_id):
    """
    Return the image_info dictionary corresponding to the specified image_id. 
    """
    all_ids = [my_dict["id"] for my_dict in image_info_list]
    index = all_ids.index(image_id)
    if image_info_list[index]:
        return image_info_list[index]
    else:
        return None
    
def draw_polygon(image_dict : dict):
    class_colors = {'Vertical_formwork': (255,0,255),
                    'Concrete_pump_hose': (0,0,255)}
    img = pyplot.imread(image_dict["path"])
    img_copy = img.copy()
    for shape in image_dict["shapes"]:
        points = np.array(tuple([x, y] for x, y in zip(shape["all_points_x"], shape["all_points_y"])))
        cv2.fillPoly(img, pts= [points], color=class_colors[shape.get('class')])
        out = cv2.addWeighted(img_copy, .6, img, 0.4, 1)
    pyplot.imshow(out)
    return out

In [ ]:
class ConstructionDataset(utils.Dataset):

    def load_images(self, dataset_dir, subset):
        """Load a subset of the construction dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. Note that we inherit the "BG" (background) class from Dataset.
        ## For now, we train our model only on polygons
        self.add_class("construction", 1, "Vertical_formwork")
        self.add_class("construction", 2, "Concrete_pump_hose")
        #self.add_class("construction", 3, "People")
        #self.add_class("construction", 4, "Mixer_truck")
        

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # We mostly care about the x and y coordinates of each region
        label_paths = sorted(glob.glob(os.path.join(dataset_dir,"labels","*json")))
        annotations = load_labels_from_folder(label_paths)
        i = 0
        for a in annotations:
            objects = a["objects"]
            shapes = objects["shape_attributes"]
            height, width = a['size']["height"], a['size']["width"]
            img_filename = ".".join(a['filename'].split(".")[:-1]) # removing the .json extension
            image_path = os.path.join(dataset_dir,"images",img_filename)
            self.add_image(
                source = "construction",
                image_id= i, #img_filename,  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                shapes=shapes)
            i +=1

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        info = filter_on_image_id(self.image_info, image_id)
        mask = np.zeros([info["height"], info["width"], len(info["shapes"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["shapes"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

In [ ]:
def train(model, epochs = 30, layers = "heads"):
    """Train the model."""
    CONSTRUCTION_DIR = "../../inputs"
    # Training dataset.
    dataset_train = ConstructionDataset()
    dataset_train.load_images(CONSTRUCTION_DIR, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = ConstructionDataset()
    dataset_val.load_images(CONSTRUCTION_DIR, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=epochs,
                layers=layers)

In [ ]:
data = ConstructionDataset()

Run the load_images method, and check that data has initialised your objects:

In [ ]:
# TO DO check shapes format in load_images
# TO DO extend labelling outside polygons

In [ ]:
data.load_images(CONSTRUCTION_DIR, "train")

Check our class instantiation. "BG" is a default class corresponding to a background object.

In [ ]:
data.class_info

Check the format of our image info.

In [ ]:
pp = pprint.PrettyPrinter()
pp.pprint(data.image_info[0])

To check the label encoding was a success, let us visualise the labels for one of the images, based on image_info: 

In [ ]:
i = random.randint(0,len(data.image_info))
test_img = data.image_info[i]

In [ ]:
pyplot.imshow(draw_polygon(test_img))

### Model (instantiation and training).

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

The model.py implementation does not work with the latest versions of tensorflow. Run the following command on your terminal to install the correct version of tensorflow:

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(
    mode="training", 
    config=config,
    model_dir=MODEL_DIR
)

### Train your model.

Choose the weights you want to initialise your model with. 

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

Changes I brought to the sourcecode: 
- [tf.Variable error linked to tensorflow version](https://github.com/matterport/Mask_RCNN/issues/1930). Changed tensorflow and keras versions to the ones in the beginning of the  notebook - This environment
- Changed number of classes to 4 in ConstructionConfig() - This notebook
- Commented multiprocessing to false in the model train method in model.py, changed the number of workers - model.py
- Changed Keras versions again before training the model? TBC


In [ ]:
#!pip install keras

In [ ]:
train(model, epochs=1, layers="heads")

### Detection

In [ ]:
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

Load the model you previously saved with the right weights and test it for inference.

In [ ]:
class InferenceConfig(ConstructionConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)
# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [ ]:
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

### Model evauation 

In [ ]:
dataset_val = ConstructionDataset()
dataset_val.load_images(CONSTRUCTION_DIR, "val")
dataset_val.prepare()

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))